# 📊 Datathon FIAP - Passos Mágicos
## Modelo Preditivo de Risco de Defasagem

Este notebook desenvolve um modelo de Machine Learning para identificar alunos em risco de defasagem educacional, utilizando os indicadores do PEDE.

**Objetivo:** Criar um modelo preditivo que identifique padrões nos indicadores que permitam prever alunos em risco antes de queda no desempenho ou aumento da defasagem.

---

## 1. Configuração do Ambiente

In [ ]:
# Montar Google Drive (executar apenas no Google Colab)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalar bibliotecas necessárias
!pip install openpyxl scikit-learn xgboost imbalanced-learn shap -q

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                             precision_score, recall_score, f1_score, roc_auc_score,
                             roc_curve, precision_recall_curve, auc)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# Deep Learning (Keras)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Configurações
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)
tf.random.set_seed(42)

print('✅ Bibliotecas importadas com sucesso!')
print(f'TensorFlow version: {tf.__version__}')

## 2. Carregamento e Preparação dos Dados

In [ ]:
# Carregar dados
CAMINHO_ARQUIVO = '/content/drive/MyDrive/Datathon/BASE_DE_DADOS_PEDE_2024_DATATHON.xlsx'

xlsx = pd.ExcelFile(CAMINHO_ARQUIVO)
df_2022 = pd.read_excel(xlsx, sheet_name='PEDE2022')
df_2023 = pd.read_excel(xlsx, sheet_name='PEDE2023')
df_2024 = pd.read_excel(xlsx, sheet_name='PEDE2024')

print(f'📊 Dados carregados:')
print(f'   PEDE 2022: {df_2022.shape[0]:,} alunos')
print(f'   PEDE 2023: {df_2023.shape[0]:,} alunos')
print(f'   PEDE 2024: {df_2024.shape[0]:,} alunos')

In [ ]:
# Função para preparar dados
def preparar_dados(df, ano):
    df_prep = df.copy()
    df_prep['ANO'] = ano
    
    col_map = {}
    for col in df_prep.columns:
        col_lower = col.lower()
        if col_lower == 'iaa': col_map[col] = 'IAA'
        elif col_lower == 'ieg' and 'destaque' not in col_lower: col_map[col] = 'IEG'
        elif col_lower == 'ips': col_map[col] = 'IPS'
        elif col_lower == 'ipp': col_map[col] = 'IPP'
        elif col_lower == 'ida' and 'destaque' not in col_lower: col_map[col] = 'IDA'
        elif col_lower == 'ipv' and 'destaque' not in col_lower: col_map[col] = 'IPV'
        elif col_lower == 'ian': col_map[col] = 'IAN'
        elif 'defas' in col_lower: col_map[col] = 'DEFASAGEM'
        elif col_lower == 'fase': col_map[col] = 'FASE'
        elif col_lower == 'gênero' or col_lower == 'genero': col_map[col] = 'GENERO'
    
    if ano == 2022:
        col_map['INDE 22'] = 'INDE'
        col_map['Pedra 22'] = 'PEDRA'
    elif ano == 2023:
        col_map['INDE 2023'] = 'INDE'
        col_map['Pedra 2023'] = 'PEDRA'
    elif ano == 2024:
        col_map['INDE 2024'] = 'INDE'
        col_map['Pedra 2024'] = 'PEDRA'
    
    df_prep = df_prep.rename(columns=col_map)
    return df_prep

# Preparar dados
df_2022_prep = preparar_dados(df_2022, 2022)
df_2023_prep = preparar_dados(df_2023, 2023)
df_2024_prep = preparar_dados(df_2024, 2024)

# Unificar
colunas = ['ANO', 'INDE', 'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN', 'DEFASAGEM', 'PEDRA', 'GENERO', 'FASE']
df_unificado = pd.concat([
    df_2022_prep[[c for c in colunas if c in df_2022_prep.columns]],
    df_2023_prep[[c for c in colunas if c in df_2023_prep.columns]],
    df_2024_prep[[c for c in colunas if c in df_2024_prep.columns]]
], ignore_index=True)

# Converter colunas numéricas
for col in ['INDE', 'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN', 'DEFASAGEM']:
    if col in df_unificado.columns:
        df_unificado[col] = pd.to_numeric(df_unificado[col], errors='coerce')

df_unificado['PEDRA'] = df_unificado['PEDRA'].replace({'Agata': 'Ágata'})

print(f'\n✅ DataFrame unificado: {len(df_unificado):,} registros')

## 3. Definição da Variável Alvo (Risco de Defasagem)

In [ ]:
# Definir variável alvo: aluno em risco de defasagem
# Critério: defasagem <= -2 anos (moderada ou severa)

df_unificado['EM_RISCO'] = (df_unificado['DEFASAGEM'] <= -2).astype(int)

print('📊 Distribuição da variável alvo (EM_RISCO):')
print(df_unificado['EM_RISCO'].value_counts())
print(f'\nPercentual em risco: {df_unificado["EM_RISCO"].mean()*100:.1f}%')

In [ ]:
# Visualizar distribuição
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Gráfico 1: Distribuição da variável alvo
cores = ['#2ecc71', '#e74c3c']
labels = ['Não em risco', 'Em risco']
valores = df_unificado['EM_RISCO'].value_counts().sort_index()
axes[0].pie(valores, labels=labels, colors=cores, autopct='%1.1f%%', startangle=90, explode=[0, 0.1])
axes[0].set_title('Distribuição da Variável Alvo', fontweight='bold')

# Gráfico 2: Risco por ano
risco_ano = df_unificado.groupby('ANO')['EM_RISCO'].mean() * 100
bars = axes[1].bar(risco_ano.index, risco_ano.values, color='#e74c3c', edgecolor='black')
axes[1].set_title('Percentual de Alunos em Risco por Ano', fontweight='bold')
axes[1].set_xlabel('Ano')
axes[1].set_ylabel('% em Risco')
for bar, val in zip(bars, risco_ano.values):
    axes[1].annotate(f'{val:.1f}%', (bar.get_x() + bar.get_width()/2, bar.get_height()),
                     textcoords='offset points', xytext=(0, 5), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('distribuicao_risco.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Feature Engineering

In [ ]:
# Selecionar features para o modelo
features_numericas = ['IDA', 'IEG', 'IAA', 'IPS', 'IPV', 'IAN']

# Verificar features disponíveis
features_disponiveis = [f for f in features_numericas if f in df_unificado.columns]
print(f'Features disponíveis: {features_disponiveis}')

# Criar dataset para modelagem
df_modelo = df_unificado[features_disponiveis + ['EM_RISCO', 'ANO', 'PEDRA']].dropna()
print(f'\nRegistros para modelagem: {len(df_modelo):,}')
print(f'Registros removidos (NaN): {len(df_unificado) - len(df_modelo):,}')

In [ ]:
# Feature Engineering adicional

# 1. Média dos indicadores
df_modelo['MEDIA_INDICADORES'] = df_modelo[features_disponiveis].mean(axis=1)

# 2. Desvio padrão dos indicadores (variabilidade)
df_modelo['STD_INDICADORES'] = df_modelo[features_disponiveis].std(axis=1)

# 3. Diferença entre IDA e IAA (gap entre desempenho real e autoavaliação)
if 'IDA' in df_modelo.columns and 'IAA' in df_modelo.columns:
    df_modelo['GAP_IDA_IAA'] = df_modelo['IDA'] - df_modelo['IAA']

# 4. Ratio IEG/IDA (engajamento relativo ao desempenho)
if 'IEG' in df_modelo.columns and 'IDA' in df_modelo.columns:
    df_modelo['RATIO_IEG_IDA'] = df_modelo['IEG'] / (df_modelo['IDA'] + 0.1)

# Atualizar lista de features
features_finais = features_disponiveis + ['MEDIA_INDICADORES', 'STD_INDICADORES', 'GAP_IDA_IAA', 'RATIO_IEG_IDA']
features_finais = [f for f in features_finais if f in df_modelo.columns]

print(f'\nFeatures finais para o modelo: {features_finais}')
print(f'Total de features: {len(features_finais)}')

In [ ]:
# Preparar X e y
X = df_modelo[features_finais]
y = df_modelo['EM_RISCO']

print(f'Shape de X: {X.shape}')
print(f'Shape de y: {y.shape}')
print(f'\nDistribuição de y:')
print(y.value_counts())

## 5. Divisão dos Dados e Balanceamento

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'Treino: {X_train.shape[0]} registros')
print(f'Teste: {X_test.shape[0]} registros')
print(f'\nDistribuição no treino:')
print(y_train.value_counts())

In [ ]:
# Normalizar features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('✅ Features normalizadas!')

In [ ]:
# Aplicar SMOTE para balancear classes (apenas no treino)
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

print(f'\nApós SMOTE:')
print(f'Treino balanceado: {X_train_balanced.shape[0]} registros')
print(f'Distribuição:')
print(pd.Series(y_train_balanced).value_counts())

## 6. Treinamento dos Modelos

### 6.1 Modelos Tradicionais (Machine Learning)

In [ ]:
# Definir modelos
modelos = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss')
}

# Treinar e avaliar modelos
resultados = {}

print('='*70)
print('TREINAMENTO E AVALIAÇÃO DOS MODELOS')
print('='*70)

for nome, modelo in modelos.items():
    print(f'\n📊 {nome}')
    print('-'*40)
    
    # Treinar
    modelo.fit(X_train_balanced, y_train_balanced)
    
    # Predições
    y_pred = modelo.predict(X_test_scaled)
    y_pred_proba = modelo.predict_proba(X_test_scaled)[:, 1]
    
    # Métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    resultados[nome] = {
        'modelo': modelo,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1_score': f1,
        'auc': auc_score,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f'Accuracy:  {acc:.4f}')
    print(f'Precision: {prec:.4f}')
    print(f'Recall:    {rec:.4f}')
    print(f'F1-Score:  {f1:.4f}')
    print(f'AUC-ROC:   {auc_score:.4f}')

### 6.2 Rede Neural (Deep Learning - Keras)

In [ ]:
# Construir modelo de Rede Neural (MLP)
def criar_modelo_mlp(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(16, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

# Criar modelo
modelo_nn = criar_modelo_mlp(X_train_balanced.shape[1])
modelo_nn.summary()

In [ ]:
# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.0001,
    verbose=1
)

# Treinar
print('\n📊 Treinando Rede Neural (MLP)...')
print('-'*40)

history = modelo_nn.fit(
    X_train_balanced, y_train_balanced,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

In [ ]:
# Avaliar Rede Neural
y_pred_nn_proba = modelo_nn.predict(X_test_scaled).flatten()
y_pred_nn = (y_pred_nn_proba > 0.5).astype(int)

acc_nn = accuracy_score(y_test, y_pred_nn)
prec_nn = precision_score(y_test, y_pred_nn)
rec_nn = recall_score(y_test, y_pred_nn)
f1_nn = f1_score(y_test, y_pred_nn)
auc_nn = roc_auc_score(y_test, y_pred_nn_proba)

resultados['Neural Network (MLP)'] = {
    'modelo': modelo_nn,
    'accuracy': acc_nn,
    'precision': prec_nn,
    'recall': rec_nn,
    'f1_score': f1_nn,
    'auc': auc_nn,
    'y_pred': y_pred_nn,
    'y_pred_proba': y_pred_nn_proba
}

print(f'\n📊 Neural Network (MLP)')
print('-'*40)
print(f'Accuracy:  {acc_nn:.4f}')
print(f'Precision: {prec_nn:.4f}')
print(f'Recall:    {rec_nn:.4f}')
print(f'F1-Score:  {f1_nn:.4f}')
print(f'AUC-ROC:   {auc_nn:.4f}')

In [ ]:
# Visualizar histórico de treinamento
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history.history['loss'], label='Treino')
axes[0].plot(history.history['val_loss'], label='Validação')
axes[0].set_title('Evolução da Loss', fontweight='bold')
axes[0].set_xlabel('Época')
axes[0].set_ylabel('Loss')
axes[0].legend()

# AUC
axes[1].plot(history.history['auc'], label='Treino')
axes[1].plot(history.history['val_auc'], label='Validação')
axes[1].set_title('Evolução do AUC', fontweight='bold')
axes[1].set_xlabel('Época')
axes[1].set_ylabel('AUC')
axes[1].legend()

plt.tight_layout()
plt.savefig('historico_treinamento_nn.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Comparação dos Modelos

In [ ]:
# Criar DataFrame com resultados
df_resultados = pd.DataFrame({
    'Modelo': list(resultados.keys()),
    'Accuracy': [r['accuracy'] for r in resultados.values()],
    'Precision': [r['precision'] for r in resultados.values()],
    'Recall': [r['recall'] for r in resultados.values()],
    'F1-Score': [r['f1_score'] for r in resultados.values()],
    'AUC-ROC': [r['auc'] for r in resultados.values()]
})

df_resultados = df_resultados.sort_values('F1-Score', ascending=False)
print('\n📊 COMPARAÇÃO DOS MODELOS')
print('='*70)
print(df_resultados.to_string(index=False))

In [ ]:
# Visualizar comparação
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Métricas por modelo
metricas = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
x = np.arange(len(df_resultados))
width = 0.15

for i, metrica in enumerate(metricas):
    axes[0].bar(x + i*width, df_resultados[metrica], width, label=metrica)

axes[0].set_title('Comparação de Métricas por Modelo', fontweight='bold')
axes[0].set_xlabel('Modelo')
axes[0].set_ylabel('Score')
axes[0].set_xticks(x + width*2)
axes[0].set_xticklabels(df_resultados['Modelo'], rotation=45, ha='right')
axes[0].legend(bbox_to_anchor=(1.02, 1))
axes[0].set_ylim(0, 1)

# Gráfico 2: Curvas ROC
for nome, res in resultados.items():
    fpr, tpr, _ = roc_curve(y_test, res['y_pred_proba'])
    axes[1].plot(fpr, tpr, label=f"{nome} (AUC={res['auc']:.3f})")

axes[1].plot([0, 1], [0, 1], 'k--', label='Random')
axes[1].set_title('Curvas ROC', fontweight='bold')
axes[1].set_xlabel('Taxa de Falsos Positivos')
axes[1].set_ylabel('Taxa de Verdadeiros Positivos')
axes[1].legend(loc='lower right', fontsize=8)

plt.tight_layout()
plt.savefig('comparacao_modelos.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Análise do Melhor Modelo

In [ ]:
# Selecionar melhor modelo (baseado em F1-Score)
melhor_modelo_nome = df_resultados.iloc[0]['Modelo']
melhor_resultado = resultados[melhor_modelo_nome]

print(f'\n🏆 MELHOR MODELO: {melhor_modelo_nome}')
print('='*50)
print(f"F1-Score: {melhor_resultado['f1_score']:.4f}")
print(f"AUC-ROC: {melhor_resultado['auc']:.4f}")

In [ ]:
# Matriz de Confusão do melhor modelo
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Matriz de confusão
cm = confusion_matrix(y_test, melhor_resultado['y_pred'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Não em Risco', 'Em Risco'],
            yticklabels=['Não em Risco', 'Em Risco'])
axes[0].set_title(f'Matriz de Confusão - {melhor_modelo_nome}', fontweight='bold')
axes[0].set_xlabel('Predito')
axes[0].set_ylabel('Real')

# Feature Importance (se disponível)
if hasattr(melhor_resultado['modelo'], 'feature_importances_'):
    importances = melhor_resultado['modelo'].feature_importances_
    indices = np.argsort(importances)[::-1]
    
    axes[1].barh(range(len(features_finais)), importances[indices], color='#3498db', edgecolor='black')
    axes[1].set_yticks(range(len(features_finais)))
    axes[1].set_yticklabels([features_finais[i] for i in indices])
    axes[1].set_title('Importância das Features', fontweight='bold')
    axes[1].set_xlabel('Importância')
else:
    # Para modelos sem feature_importances_, usar coeficientes ou outra métrica
    axes[1].text(0.5, 0.5, 'Feature importance não disponível\npara este modelo', 
                 ha='center', va='center', fontsize=12)
    axes[1].set_title('Importância das Features', fontweight='bold')

plt.tight_layout()
plt.savefig('analise_melhor_modelo.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Relatório de classificação detalhado
print(f'\n📊 Relatório de Classificação - {melhor_modelo_nome}')
print('='*60)
print(classification_report(y_test, melhor_resultado['y_pred'], 
                           target_names=['Não em Risco', 'Em Risco']))

## 9. Salvar Modelo para Deploy

In [ ]:
import joblib

# Salvar o melhor modelo tradicional (Random Forest ou XGBoost)
# Escolher o melhor modelo tradicional para o Streamlit
modelo_para_deploy = resultados['Random Forest']['modelo']  # ou XGBoost

# Salvar modelo
joblib.dump(modelo_para_deploy, 'modelo_risco_defasagem.pkl')
print('✅ Modelo salvo: modelo_risco_defasagem.pkl')

# Salvar scaler
joblib.dump(scaler, 'scaler.pkl')
print('✅ Scaler salvo: scaler.pkl')

# Salvar lista de features
with open('features.txt', 'w') as f:
    f.write('\n'.join(features_finais))
print('✅ Features salvas: features.txt')

In [ ]:
# Salvar modelo Keras (opcional)
modelo_nn.save('modelo_nn_risco_defasagem.h5')
print('✅ Modelo Neural Network salvo: modelo_nn_risco_defasagem.h5')

## 10. Função de Predição para Novos Alunos

In [ ]:
def prever_risco(ida, ieg, iaa, ips, ipv, ian):
    """
    Função para prever risco de defasagem de um aluno.
    
    Parâmetros:
    - ida: Indicador de Desempenho Acadêmico (0-10)
    - ieg: Indicador de Engajamento (0-10)
    - iaa: Indicador de Autoavaliação (0-10)
    - ips: Indicador Psicossocial (0-10)
    - ipv: Indicador de Ponto de Virada (0-10)
    - ian: Indicador de Adequação ao Nível (0-10)
    
    Retorna:
    - Probabilidade de estar em risco (0-1)
    - Classificação (Em Risco / Não em Risco)
    """
    # Criar features
    media_ind = np.mean([ida, ieg, iaa, ips, ipv, ian])
    std_ind = np.std([ida, ieg, iaa, ips, ipv, ian])
    gap_ida_iaa = ida - iaa
    ratio_ieg_ida = ieg / (ida + 0.1)
    
    # Criar array de features
    features = np.array([[ida, ieg, iaa, ips, ipv, ian, media_ind, std_ind, gap_ida_iaa, ratio_ieg_ida]])
    
    # Normalizar
    features_scaled = scaler.transform(features)
    
    # Predição
    prob = modelo_para_deploy.predict_proba(features_scaled)[0, 1]
    classificacao = 'Em Risco' if prob > 0.5 else 'Não em Risco'
    
    return prob, classificacao

# Exemplo de uso
print('\n📊 EXEMPLO DE PREDIÇÃO')
print('='*50)

# Aluno exemplo 1 (baixo risco)
prob1, class1 = prever_risco(ida=8.0, ieg=8.5, iaa=8.0, ips=7.5, ipv=7.0, ian=8.0)
print(f'\nAluno 1 (bom desempenho):')
print(f'  Probabilidade de risco: {prob1:.2%}')
print(f'  Classificação: {class1}')

# Aluno exemplo 2 (alto risco)
prob2, class2 = prever_risco(ida=4.0, ieg=3.5, iaa=5.0, ips=4.0, ipv=3.0, ian=4.0)
print(f'\nAluno 2 (baixo desempenho):')
print(f'  Probabilidade de risco: {prob2:.2%}')
print(f'  Classificação: {class2}')

---

## 📝 Conclusões

### Principais Descobertas:

1. **Melhor Modelo:** O modelo com melhor desempenho foi identificado com base no F1-Score, equilibrando precisão e recall.

2. **Features Mais Importantes:** Os indicadores que mais contribuem para identificar alunos em risco são tipicamente IDA, IEG e IAN.

3. **Desempenho:** O modelo consegue identificar alunos em risco com boa acurácia, permitindo intervenções preventivas.

### Recomendações:

- Utilizar o modelo para identificação precoce de alunos em risco
- Focar intervenções em alunos com alta probabilidade de risco
- Monitorar especialmente os indicadores IDA e IEG
- Atualizar o modelo periodicamente com novos dados

---